In [1]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 18.2MB/s 
     |████████████████████████████████| 1.2MB 44.6MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 194kB 57.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 776.8MB 21kB/s 
     |████████████████████████████████| 12.8MB 247kB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

# NLP Deep Dive: RNNs

## Text Preprocessing

### Tokenization

### Word Tokenization with fastai

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
path.ls()

(#7) [Path('/root/.fastai/data/imdb/test'),Path('/root/.fastai/data/imdb/imdb.vocab'),Path('/root/.fastai/data/imdb/README'),Path('/root/.fastai/data/imdb/tmp_lm'),Path('/root/.fastai/data/imdb/unsup'),Path('/root/.fastai/data/imdb/train'),Path('/root/.fastai/data/imdb/tmp_clas')]

In [5]:
get_text_files?

Only check in given folders.

In [6]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [7]:
files[0]

Path('/root/.fastai/data/imdb/test/pos/4814_7.txt')

Even inside test, folders with neg pos labels.

In [8]:
txt = files[0].open().read(); 
txt[:75]

'The late Director John Frankenheimer directed his first feature film, The Y'

In [9]:
txt

"The late Director John Frankenheimer directed his first feature film, The Young Stranger, after starting out directing live television dramas in New York City. This film came on the heels of the success of Rebel Without A Cause in 1955. James MacArthur made his feature film debut as a troubled teen with a movie producer father, played by James Daly, who doesn't establish enough of a relationship with his son. Kim Hunter plays the mother, who tries to bridge the gap between her husband and her son. The film uses the popular juvenile delinquent angle of the time to tell its story. MacArthur gets in trouble at a movie theater with an overzealous theater manager played by Whit Bissell. MacArthur, in turn, has to deal with a police sergeant, James Gregory, bent on teaching him a lesson. The material could easily have turned exploitative, laughable, and sensational, like any number of others of the period did. However, under the sure-handed direction of Frankenheimer, the film is a sensitiv

In [10]:
w_tkn = WordTokenizer()

In [11]:
first?

In [12]:
tokens = first(w_tkn([txt])) # needs a list

In [13]:
coll_repr(tokens,10)

"(#286) ['The','late','Director','John','Frankenheimer','directed','his','first','feature','film'...]"

In [14]:
coll_repr??

In [15]:
w_tkn = WordTokenizer()
tkn = Tokenizer(w_tkn) # from fastai

In [16]:
tokens = tkn(txt)

In [17]:
print(coll_repr(tokens,10))

(#321) ['xxbos','xxmaj','the','late','xxmaj','director','xxmaj','john','xxmaj','frankenheimer'...]


In [18]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [19]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

### Subword Tokenization

In [20]:
txts = L(o.open().read() for o in files[:2000])

In [21]:
txts

(#2000) ["The late Director John Frankenheimer directed his first feature film, The Young Stranger, after starting out directing live television dramas in New York City. This film came on the heels of the success of Rebel Without A Cause in 1955. James MacArthur made his feature film debut as a troubled teen with a movie producer father, played by James Daly, who doesn't establish enough of a relationship with his son. Kim Hunter plays the mother, who tries to bridge the gap between her husband and her son. The film uses the popular juvenile delinquent angle of the time to tell its story. MacArthur gets in trouble at a movie theater with an overzealous theater manager played by Whit Bissell. MacArthur, in turn, has to deal with a police sergeant, James Gregory, bent on teaching him a lesson. The material could easily have turned exploitative, laughable, and sensational, like any number of others of the period did. However, under the sure-handed direction of Frankenheimer, the film is a

In [22]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [23]:
subword(1000)

'▁The ▁la te ▁D ir ec t or ▁John ▁Fr ank en he im er ▁direct ed ▁his ▁first ▁feature ▁film , ▁The ▁You n g ▁St range r , ▁after ▁start ing ▁out ▁direct ing ▁live ▁ te le'

In [24]:
subword(200)

'▁The ▁ la t e ▁ D i re c t or ▁ J o h n ▁ F r an k en h e i m er ▁d i re c t ed ▁h is ▁f ir st ▁f'

In [25]:
subword(10000)

'▁The ▁late ▁Director ▁John ▁Frankenheimer ▁directed ▁his ▁first ▁feature ▁film , ▁The ▁Young ▁ Stranger , ▁after ▁start ing ▁out ▁directing ▁live ▁television ▁dramas ▁in ▁New ▁York ▁City . ▁This ▁film ▁came ▁on ▁the ▁he el s ▁of ▁the ▁success'

Larger vocab size means more vocabulary can be found. Hence more sentences can be sub-word tokenized. 
But fewer tokens per sentence. 

### Numericalization with fastai

In [26]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#321) ['xxbos','xxmaj','the','late','xxmaj','director','xxmaj','john','xxmaj','frankenheimer','directed','his','first','feature','film',',','xxmaj','the','xxmaj','young','xxmaj','stranger',',','after','starting','out','directing','live','television','dramas','in'...]


In [27]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#321) ['xxbos','xxmaj','the','late','xxmaj','director','xxmaj','john','xxmaj','frankenheimer'...]

In [28]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2136) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [29]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,    9,  380,    8,  180,    8,  215,    8, 1558,  597,   37,  109, 1249,   31,   10,    8,    9,    8,  183])

In [30]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj the late xxmaj director xxmaj john xxmaj frankenheimer directed his first feature film , xxmaj the xxmaj young'

### Putting Our Texts into Batches for a Language Model

In [31]:
tokens = tkn(txt)
bs, seq_len = 3,6

In [32]:
coll_repr(tokens,100)

'(#321) [\'xxbos\',\'xxmaj\',\'the\',\'late\',\'xxmaj\',\'director\',\'xxmaj\',\'john\',\'xxmaj\',\'frankenheimer\',\'directed\',\'his\',\'first\',\'feature\',\'film\',\',\',\'xxmaj\',\'the\',\'xxmaj\',\'young\',\'xxmaj\',\'stranger\',\',\',\'after\',\'starting\',\'out\',\'directing\',\'live\',\'television\',\'dramas\',\'in\',\'xxmaj\',\'new\',\'xxmaj\',\'york\',\'xxmaj\',\'city\',\'.\',\'xxmaj\',\'this\',\'film\',\'came\',\'on\',\'the\',\'heels\',\'of\',\'the\',\'success\',\'of\',\'xxmaj\',\'rebel\',\'xxmaj\',\'without\',\'a\',\'xxmaj\',\'cause\',\'in\',\'1955\',\'.\',\'xxmaj\',\'james\',\'macarthur\',\'made\',\'his\',\'feature\',\'film\',\'debut\',\'as\',\'a\',\'troubled\',\'teen\',\'with\',\'a\',\'movie\',\'producer\',\'father\',\',\',\'played\',\'by\',\'xxmaj\',\'james\',\'xxmaj\',\'daly\',\',\',\'who\',\'does\',"n\'t",\'establish\',\'enough\',\'of\',\'a\',\'relationship\',\'with\',\'his\',\'son\',\'.\',\'xxmaj\',\'kim\',\'xxmaj\',\'hunter\'...]'

make a batch

In [33]:
i =0 ;
coll_repr(tokens[i*seq_len:(i+1)*seq_len],100) # to select from pos 0 to 15, 15 to 30, etc.. i+1 is to correct for indices  python

"(#6) ['xxbos','xxmaj','the','late','xxmaj','director']"

In [34]:
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)]) # rows i*seq_len, making them separate lists.

In [35]:
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,the,late,xxmaj,director
xxmaj,john,xxmaj,frankenheimer,directed,his
first,feature,film,",",xxmaj,the


In [36]:
nums200 = toks200.map(num)

In [37]:
dl = LMDataLoader(nums200)

In [38]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [39]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj the late xxmaj director xxmaj john xxmaj frankenheimer directed his first feature film , xxmaj the xxmaj young'

In [40]:
' '.join(num.vocab[o] for o in y[0][:20]) # depend has one extra word, as we are trying to predict that.

'xxmaj the late xxmaj director xxmaj john xxmaj frankenheimer directed his first feature film , xxmaj the xxmaj young xxmaj'

## Training a Text Classifier

### Language Model Using DataBlock

In [41]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])
get_imdb

functools.partial(<function get_text_files at 0x7f4121605680>, folders=['train', 'test', 'unsup'])

In [42]:
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.2)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [43]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj this film does not have the outstanding visuals that xxmaj american xxmaj beauty or xxmaj the xxmaj ice xxmaj storm offered and because it was made after those films , it can not be marked as very original either : the music , though subtly brilliant , sounds very much like that of xxmaj american xxmaj beauty . \n\n xxmaj the story has some similarities with xxmaj the xxmaj ice xxmaj storm in particular ( as well as","xxmaj this film does not have the outstanding visuals that xxmaj american xxmaj beauty or xxmaj the xxmaj ice xxmaj storm offered and because it was made after those films , it can not be marked as very original either : the music , though subtly brilliant , sounds very much like that of xxmaj american xxmaj beauty . \n\n xxmaj the story has some similarities with xxmaj the xxmaj ice xxmaj storm in particular ( as well as xxmaj"
1,"that you spend 30 % of your life watching , because you ca n't help yourself … especially when you have nothing to do late at night except to channel surf and watch b movies … \n\n xxmaj good premise of a film , got me interested , forgave the cheesy acting , did n't mind the simple xxup fx , but the script was real bad . i could have cried . \n\n xxmaj if you have nothing better","you spend 30 % of your life watching , because you ca n't help yourself … especially when you have nothing to do late at night except to channel surf and watch b movies … \n\n xxmaj good premise of a film , got me interested , forgave the cheesy acting , did n't mind the simple xxup fx , but the script was real bad . i could have cried . \n\n xxmaj if you have nothing better to"


### Fine-Tuning the Language Model

In [80]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [81]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time


RuntimeError: ignored

### Saving and Loading Models

In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

### Text Generation

In [82]:
learn = learn.load_encoder('finetuned')

In [83]:
prompt = "Aliens are"
learn.predict(prompt)

'Aliens are researching'

In [84]:
prompt = "How can we see this as a "
N_WORDS = 100
N_SENTENCES = 2
preds = [learn.predict(prompt, N_WORDS, temperature=0.75)  for _ in range(N_SENTENCES)]
print("\n".join(preds))

How can we see this as a great war movie . Walter Matthau plays a rookie agent who is assigned to an assignment with new superintendent : reporting : they have three music videos , " goodbye to Reading days " ( sung by the lead singer ) which pays homage to veteran dancer Gene Kelly ( who voiced his philosophy ) , and three lovely partners who accompany him on parole ; among them are ex soldier Kelly and surgeon Felix Ungar ( played by brother ) who has a enamored associate named h mmm [[[
How can we see this as a warning warning against invaders from such lands today ? It was filmed in 1995 ! ! 

 It reminded me of how planes distributed animals are compared to civilian birds . Yes , some butch birds were killed later on after midnight but in 1996 they came to photograph snakes running around in their fur coats on ill - condition looking " lizards " . Productions nowadays confirmed this theory simply using rubber snakes creating awful accents . Another instance : every now and then a 

### Creating the Classifier DataLoaders

In [51]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=get_imdb,
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [52]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj some have praised xxunk xxmaj lost xxmaj xxunk as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think xxmaj the xxmaj dirty xxmaj dozen ) but xxunk represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before ,",neg
2,"xxbos xxmaj jim xxmaj carrey is back to much the same role that he played in xxmaj the xxmaj mask , a timid guy who is trying to get ahead in the world but who seems to be plagued with bad luck . xxmaj even when he tries to help a homeless guy from being harassed by a bunch of hoodlums ( and of course they have to be xxmaj mexican , obviously ) , his good will towards his fellow man backfires . xxmaj in that case , it was n't too hard to predict that he was about to have a handful of angry hoodlums , but i like that the movie suggests that things like that should n't be ignored . xxmaj i 'm reminded of the episode of xxmaj michael xxmaj moore 's brilliant xxmaj the xxmaj awful xxmaj truth , when they had a man",pos


In [53]:
nums_samp = toks200[:10].map(num)

In [54]:
nums_samp.map(len)

(#10) [321,98,259,151,256,571,185,367,99,97]

In [55]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [56]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

In [57]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.253200,0.184729,0.929000,01:10


In [58]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.225904,0.169303,0.935480,01:15


In [59]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.190461,0.150287,0.944080,01:33


In [60]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.160430,0.154835,0.943560,01:52
1,0.149770,0.148728,0.945560,01:53


## Disinformation and Language Models

## Conclusion

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?